# Compile pattern data only for POI near stations
This file loads the compiled patterns data, station list, and json files of the nearby POIs to each station, then calculates the average patterns for nearby POIs to each station.

In [ ]:
savedate = '20250401'
root = ''
folder1 = root + 'Data/SafeGraphData/DownloadedData/2022/10/MP/'
folder2 = root + 'Data/SafeGraphData/ProcessedData/final_data_250330/'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import geopandas
import json
import time

### Load data

In [ ]:
pattern_data = pd.read_csv(folder1 + 'compiled_patterns.csv', index_col=0)
station_id_df = pd.read_csv(folder2 + '00_alt_fuel_stations (Apr 3 2023)_wFIPS.csv', index_col=0).reset_index()
station_id_df = station_id_df.loc[:, ['unique_ID', 'State']]
state_names = list(station_id_df['State'].unique())

In [ ]:
nearbypois_500m = {}
nearbypois_10closest = {}
ct_nopatterns = 0

combined_df_stations = None

for state in np.sort(state_names):
    tic = time.time()
    print('---'*5, state, '---'*5)
    filename = folder2 + 'stations_PoI_uniqueID_within_500m/'+state+'_500.json'
    nearbypois_500m[state] = json.load(open(filename))
    filename = folder2 + '10neareststations_PoI_uniqueID_within_5000m/'+state+'_5000.json'
    nearbypois_10closest[state] = json.load(open(filename))

    station_id_df_here = station_id_df.loc[station_id_df['State']==state].copy().reset_index(drop=True)
    
    for idx in station_id_df_here.index:
        key = station_id_df_here.loc[idx, 'unique_ID']
        if len(nearbypois_500m[state][key]) >= 10:
            poi_subset = pattern_data.loc[pattern_data['placekey'].isin(nearbypois_500m[state][key])]
        else:
            poi_subset = pattern_data.loc[pattern_data['placekey'].isin(nearbypois_10closest[state][key])]
        means = poi_subset.mean(axis=0, numeric_only=True)
        for key1, val in means.items():
            station_id_df_here.loc[idx, key1] = val
            
    print('Nans:', np.round(station_id_df_here['distance_from_home'].isna().sum() / len(station_id_df_here), 4))
    
    if combined_df_stations is None:
        combined_df_stations = station_id_df_here.copy(deep=True)
    else:
        combined_df_stations = pd.concat((combined_df_stations, station_id_df_here.copy(deep=True)), axis=0, ignore_index=True)
        
    toc = time.time()
    print('Time for ', state, ' in minutes: ', np.round((1/60)*(toc-tic), 4))
    combined_df_stations.to_csv(folder2 + 'stationlevel/patternsonly_uniqueID_'+savedate+'.csv')

### Stats

In [ ]:
combined_df_stations['distance_from_home'].isna().sum() / len(combined_df_stations)

In [ ]:
combined_df_stations['State'].value_counts()

In [ ]:
combined_df_stations.isna().sum()